In [ ]:
!pip install qdrant_client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 9.2 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import load_dataset
from transformers import CLIPProcessor, CLIPModel
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import PointStruct
import uuid
import math

In [ ]:
MODEL_NAME = "openai/clip-vit-base-patch32"
DATASET_NAME = "ashraq/fashion-product-images-small"
COLLECTION_NAME = "fashion_products_poc" # Tên collection trên Qdrant

SAMPLE_SIZE = 1000  # Số lượng mẫu để làm PoC (lấy 1000 cho nhanh)
BATCH_SIZE = 64     # Xử lý 64 ảnh/lần

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Đang sử dụng thiết bị: {device}")

Đang sử dụng thiết bị: cpu


In [ ]:
model = CLIPModel.from_pretrained(MODEL_NAME).to(device)
processor = CLIPProcessor.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
# location=":memory:" có nghĩa là CSDL chạy hoàn toàn trên RAM
# và sẽ bị xóa khi script kết thúc.
print("Khởi động Qdrant Client (in-memory)...")
client = QdrantClient(location=":memory:")

Khởi động Qdrant Client (in-memory)...


In [ ]:
vector_size = model.config.projection_dim
print(f"Kích thước Vector: {vector_size}")

Kích thước Vector: 512


In [ ]:
# Tạo Collection trên Qdrant
try:
    client.recreate_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=models.VectorParams(
            size=vector_size,
            distance=models.Distance.COSINE # Dùng Cosine Similarity
        )
    )
    print(f"Collection '{COLLECTION_NAME}' đã được tạo.")
except Exception as e:
    print(f"Không thể tạo collection: {e}")
    exit()

Collection 'fashion_products_poc' đã được tạo.


/tmp/ipython-input-3897730014.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [ ]:
dataset = load_dataset(DATASET_NAME, split=f'train[:{SAMPLE_SIZE}]')

README.md:   0%|          | 0.00/867 [00:00<?, ?B/s]

data/train-00000-of-00002-6cff4c59f91661(…):   0%|          | 0.00/136M [00:00<?, ?B/s]

data/train-00001-of-00002-bb459e5ac5f01e(…):   0%|          | 0.00/135M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44072 [00:00<?, ? examples/s]

In [ ]:
print("\n--- Bắt đầu Giai đoạn 2: Lập chỉ mục ---")
total_batches = math.ceil(SAMPLE_SIZE / BATCH_SIZE)
successful_uploads = 0

for i in range(total_batches):
    start_idx = i * BATCH_SIZE
    end_idx = min((i + 1) * BATCH_SIZE, SAMPLE_SIZE)

    print(f"Đang xử lý lô (batch) {i+1}/{total_batches} (mẫu {start_idx} đến {end_idx})...")

    batch_data_raw = dataset[start_idx:end_idx]
    num_items_in_batch = len(batch_data_raw['id'])

    # --- PHẦN LỌC DỮ LIỆU ---
    valid_images = []
    valid_ids = []
    valid_payloads = []

    for j in range(num_items_in_batch):

        img = batch_data_raw['image'][j]

        # **SỬA LỖI: Giữ ID là SỐ NGUYÊN (INTEGER)**
        item_id = batch_data_raw['id'][j]

        if img:
            try:
                img_rgb = img.convert("RGB")

                valid_images.append(img_rgb)
                valid_ids.append(item_id) # ID giờ là số nguyên
                valid_payloads.append({
                    "product_name": batch_data_raw['productDisplayName'][j],
                    "category": batch_data_raw['articleType'][j],
                    "gender": batch_data_raw['gender'][j],
                    "base_color": batch_data_raw['baseColour'][j],
                    "original_id": item_id
                })
            except Exception as e:
                print(f"  > Lỗi convert ảnh (ID: {item_id}): {e}. Bỏ qua ảnh này.")

    if not valid_images:
        print("  > Cả lô không có ảnh hợp lệ. Bỏ qua.")
        continue

    # --- KẾT THÚC PHẦN LỌC ---

    try:
        # 1. Mã hóa Hình ảnh
        inputs = processor(
            images=valid_images,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(device)

        with torch.no_grad():
            image_vectors = model.get_image_features(**inputs)

        # 2. Chuẩn bị "Points"
        points_to_upsert = [
            PointStruct(
                id=valid_ids[k], # Qdrant sẽ nhận ID là số nguyên
                vector=image_vectors[k].cpu().numpy().tolist(),
                payload=valid_payloads[k]
            )
            for k in range(len(valid_ids))
        ]

        # 3. Lưu (Upsert) vào Qdrant
        client.upsert(
            collection_name=COLLECTION_NAME,
            points=points_to_upsert,
            wait=True
        )

        successful_uploads += len(valid_ids)

    except Exception as e:
        print(f"Lỗi khi Upsert lô {i+1} lên Qdrant: {e}")
        print(f"  > Lỗi ID. IDs trong lô này: {valid_ids}")

# Kiểm tra lại số lượng
print("\n--- HOÀN TẤT LẬP CHỈ MỤC ---")
print(f"Tổng số vector đã được lập chỉ mục thành công: {successful_uploads}")
count_result = client.count(collection_name=COLLECTION_NAME, exact=True)
print(f"Số lượng Qdrant ghi nhận: {count_result.count}")


--- Bắt đầu Giai đoạn 2: Lập chỉ mục ---
Đang xử lý lô (batch) 1/16 (mẫu 0 đến 64)...
Đang xử lý lô (batch) 2/16 (mẫu 64 đến 128)...
Đang xử lý lô (batch) 3/16 (mẫu 128 đến 192)...
Đang xử lý lô (batch) 4/16 (mẫu 192 đến 256)...
Đang xử lý lô (batch) 5/16 (mẫu 256 đến 320)...
Đang xử lý lô (batch) 6/16 (mẫu 320 đến 384)...
Đang xử lý lô (batch) 7/16 (mẫu 384 đến 448)...
Đang xử lý lô (batch) 8/16 (mẫu 448 đến 512)...
Đang xử lý lô (batch) 9/16 (mẫu 512 đến 576)...
Đang xử lý lô (batch) 10/16 (mẫu 576 đến 640)...
Đang xử lý lô (batch) 11/16 (mẫu 640 đến 704)...
Đang xử lý lô (batch) 12/16 (mẫu 704 đến 768)...
Đang xử lý lô (batch) 13/16 (mẫu 768 đến 832)...
Đang xử lý lô (batch) 14/16 (mẫu 832 đến 896)...
Đang xử lý lô (batch) 15/16 (mẫu 896 đến 960)...
Đang xử lý lô (batch) 16/16 (mẫu 960 đến 1000)...

--- HOÀN TẤT LẬP CHỈ MỤC ---
Tổng số vector đã được lập chỉ mục thành công: 1000
Số lượng Qdrant ghi nhận: 1000


In [ ]:
print("--- Bắt đầu Giai đoạn 3: Tìm kiếm ---")
def search_fashion_items(query_text, top_k=5):
    """
    Hàm này nhận một chuỗi văn bản, mã hóa nó bằng CLIP,
    và tìm kiếm trong Qdrant để trả về các kết quả gần nhất.
    """
    print(f"\n🔍 Đang tìm kiếm cho: '{query_text}'...")

    try:
        # 1. Mã hóa truy vấn văn bản (Text -> Vector)
        inputs = processor(
            text=[query_text],
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(device)

        with torch.no_grad():
            text_vector = model.get_text_features(**inputs)

        # Chuyển đổi vector sang định dạng list
        query_vector = text_vector[0].cpu().numpy().tolist()

        # 2. Thực hiện tìm kiếm trên Qdrant (ĐÃ SỬA CHO API MỚI)
        response = client.query_points(
            collection_name=COLLECTION_NAME,
            query=query_vector,
            limit=top_k,
            with_payload=True
        )

        search_results = response.points

        # 3. Hiển thị kết quả
        print("✅ Kết quả tìm kiếm:")
        if not search_results:
            print("  > Không tìm thấy kết quả phù hợp.")
            return

        for i, result in enumerate(search_results):
            print(f"  {i+1}. ID: {result.id}")
            print(f"      Điểm tương đồng (Score): {result.score:.4f}")
            print(f"      Tên SP: {result.payload.get('product_name')}")
            print(f"      Loại: {result.payload.get('category')}")
            print(f"      Giới tính: {result.payload.get('gender')}")
            print(f"      Màu: {result.payload.get('base_color')}")

    except Exception as e:
        print(f"Lỗi khi tìm kiếm: {e}")
        # In thêm chi tiết để debug nếu cần
        import traceback
        traceback.print_exc()

--- Bắt đầu Giai đoạn 3: Tìm kiếm ---


In [ ]:
# Kiểm tra xem collection có dữ liệu không
if successful_uploads > 0:
    search_fashion_items("blue shirt for men")
    search_fashion_items("red high heels")
    search_fashion_items("black wallet")
    search_fashion_items("a person wearing sunglasses")
else:
    print("Bỏ qua tìm kiếm vì không có dữ liệu nào được lập chỉ mục.")


🔍 Đang tìm kiếm cho: 'blue shirt for men'...
✅ Kết quả tìm kiếm:
  1. ID: 13417
      Điểm tương đồng (Score): 0.3276
      Tên SP: United Colors of Benetton Men Solid Blue Shirts
      Loại: Shirts
      Giới tính: Men
      Màu: Blue
  2. ID: 19540
      Điểm tương đồng (Score): 0.3251
      Tên SP: United Colors of Benetton Men Solid Blue Sweater
      Loại: Sweaters
      Giới tính: Men
      Màu: Blue
  3. ID: 11712
      Điểm tương đồng (Score): 0.3237
      Tên SP: Lee Men Solid Blue Shirts
      Loại: Shirts
      Giới tính: Men
      Màu: Blue
  4. ID: 59299
      Điểm tương đồng (Score): 0.3128
      Tên SP: U.S. Polo Assn. Men Striped Blue Shirt
      Loại: Shirts
      Giới tính: Men
      Màu: Blue
  5. ID: 2272
      Điểm tương đồng (Score): 0.3120
      Tên SP: Nike Mens Blue Polo T-shirt
      Loại: Tshirts
      Giới tính: Men
      Màu: Blue

🔍 Đang tìm kiếm cho: 'red high heels'...
✅ Kết quả tìm kiếm:
  1. ID: 43367
      Điểm tương đồng (Score): 0.2479
      Tên SP